In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import datatable as dt
import sklearn
import xgboost as xgb
import optuna 
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.utils.validation import _deprecate_positional_args
from sklearn.model_selection._split import _BaseKFold, indexable, _num_samples



# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Getting data and get rid of weight == 0, setting scaler

In [ ]:
train_data_datatable = dt.fread('/kaggle/input/jane-street-market-prediction/train.csv')
train_data = train_data_datatable.to_pandas()
train_data.set_index('ts_id', inplace = True)
del train_data_datatable
train_data = train_data[train_data['weight'] != 0]
scaler = StandardScaler()

Fillforward missing data and dropping duplicates

In [ ]:
imputed_train_df = train_data.ffill(axis = 0)
imputed_train_df.drop_duplicates(keep=False,inplace=True)
imputed_train_df = imputed_train_df.dropna()
del train_data

create the colum 'action' and split into training set and cross validation set

In [ ]:
imputed_train_df['action'] = ((imputed_train_df['weight'].values * imputed_train_df['resp'].values) > 0).astype('int')
dlabels = imputed_train_df[['date', 'weight', 'resp_1', 'resp_2', 'resp_3', 'resp_4', 'resp']]
X_train = imputed_train_df.loc[:, imputed_train_df.columns.str.contains('feature')]
y = imputed_train_df.loc[:, ['action', 'resp', 'weight', 'date']]
groups = imputed_train_df.loc[:, 'date'].values
del imputed_train_df

PCA

In [ ]:
scaler.fit(X_train.values)
train_x_norm = scaler.transform(X_train.values)

pca = PCA()
comp = pca.fit(train_x_norm)

pca = PCA(n_components=70).fit(train_x_norm)
X = pca.transform(train_x_norm)
del train_x_norm

PurgedGroupTimeSeriesSplit class

In [ ]:
# modified code for group gaps; source
# https://github.com/getgaurav2/scikit-learn/blob/d4a3af5cc9da3a76f0266932644b884c99724c57/sklearn/model_selection/_split.py#L2243
class PurgedGroupTimeSeriesSplit(_BaseKFold):
    """Time Series cross-validator variant with non-overlapping groups.
    Allows for a gap in groups to avoid potentially leaking info from
    train into test if the model has windowed or lag features.
    Provides train/test indices to split time series data samples
    that are observed at fixed time intervals according to a
    third-party provided group.
    In each split, test indices must be higher than before, and thus shuffling
    in cross validator is inappropriate.
    This cross-validation object is a variation of :class:`KFold`.
    In the kth split, it returns first k folds as train set and the
    (k+1)th fold as test set.
    The same group will not appear in two different folds (the number of
    distinct groups has to be at least equal to the number of folds).
    Note that unlike standard cross-validation methods, successive
    training sets are supersets of those that come before them.
    Read more in the :ref:`User Guide <cross_validation>`.
    Parameters
    ----------
    n_splits : int, default=5
        Number of splits. Must be at least 2.
    max_train_group_size : int, default=Inf
        Maximum group size for a single training set.
    group_gap : int, default=None
        Gap between train and test
    max_test_group_size : int, default=Inf
        We discard this number of groups from the end of each train split
    """

    @_deprecate_positional_args
    def __init__(self,
                 n_splits=5,
                 *,
                 max_train_group_size=np.inf,
                 max_test_group_size=np.inf,
                 group_gap=None,
                 verbose=False
                 ):
        super().__init__(n_splits, shuffle=False, random_state=None)
        self.max_train_group_size = max_train_group_size
        self.group_gap = group_gap
        self.max_test_group_size = max_test_group_size
        self.verbose = verbose

    def split(self, X, y=None, groups=None):
        """Generate indices to split data into training and test set.
        Parameters
        ----------
        X : array-like of shape (n_samples, n_features)
            Training data, where n_samples is the number of samples
            and n_features is the number of features.
        y : array-like of shape (n_samples,)
            Always ignored, exists for compatibility.
        groups : array-like of shape (n_samples,)
            Group labels for the samples used while splitting the dataset into
            train/test set.
        Yields
        ------
        train : ndarray
            The training set indices for that split.
        test : ndarray
            The testing set indices for that split.
        """
        if groups is None:
            raise ValueError(
                "The 'groups' parameter should not be None")
        X, y, groups = indexable(X, y, groups)
        n_samples = _num_samples(X)
        n_splits = self.n_splits
        group_gap = self.group_gap
        max_test_group_size = self.max_test_group_size
        max_train_group_size = self.max_train_group_size
        n_folds = n_splits + 1
        group_dict = {}
        u, ind = np.unique(groups, return_index=True)
        unique_groups = u[np.argsort(ind)]
        n_samples = _num_samples(X)
        n_groups = _num_samples(unique_groups)
        for idx in np.arange(n_samples):
            if (groups[idx] in group_dict):
                group_dict[groups[idx]].append(idx)
            else:
                group_dict[groups[idx]] = [idx]
        if n_folds > n_groups:
            raise ValueError(
                ("Cannot have number of folds={0} greater than"
                 " the number of groups={1}").format(n_folds,
                                                     n_groups))

        group_test_size = min(n_groups // n_folds, max_test_group_size)
        group_test_starts = range(n_groups - n_splits * group_test_size,
                                  n_groups, group_test_size)
        for group_test_start in group_test_starts:
            train_array = []
            test_array = []

            group_st = max(0, group_test_start - group_gap - max_train_group_size)
            for train_group_idx in unique_groups[group_st:(group_test_start - group_gap)]:
                train_array_tmp = group_dict[train_group_idx]
                
                train_array = np.sort(np.unique(
                                      np.concatenate((train_array,
                                                      train_array_tmp)),
                                      axis=None), axis=None)

            train_end = train_array.size
 
            for test_group_idx in unique_groups[group_test_start:
                                                group_test_start +
                                                group_test_size]:
                test_array_tmp = group_dict[test_group_idx]
                test_array = np.sort(np.unique(
                                              np.concatenate((test_array,
                                                              test_array_tmp)),
                                     axis=None), axis=None)

            test_array  = test_array[group_gap:]
            
            
            if self.verbose > 0:
                    pass
                    
            yield [int(i) for i in train_array], [int(i) for i in test_array]

utility function to optimize in CV

In [ ]:
def utility(estimator, X, date, weight, resp):
    """Custom scoring object as per documentation:
    https://scikit-learn.org/stable/modules/model_evaluation.html#implementing-your-own-scoring-object
    Utility score formulae are defined in competition's intro:
    https://www.kaggle.com/c/jane-street-market-prediction/overview/evaluation
    Using optimisation tricks from @gogo827jz:
    https://www.kaggle.com/c/jane-street-market-prediction/discussion/201257
    """

    action = np.rint(estimator.predict(X))
        
    p_i = np.bincount(date, weight * resp * action)
    t = p_i.sum() / np.sqrt((p_i ** 2).sum()) * np.sqrt(250 / np.unique(date).size)
    u = np.clip(t, 0, 6) * p_i.sum()
    return u

function to optimize with cross validation. Applying PurgedGroupTimeSeriesSplit

In [ ]:
cv = PurgedGroupTimeSeriesSplit(
    n_splits=3,
    max_train_group_size=150,
    group_gap=20,
    max_test_group_size=60
)

def objective(trial, cv=cv, cv_fold_func=np.average):

    # Optuna suggest params
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 350, 1000),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'learning_rate': trial.suggest_uniform('learning_rate', 0.01, 0.10),
        'subsample': trial.suggest_uniform('subsample', 0.50, 0.90),
        'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.50, 0.90),
        'gamma': trial.suggest_int('gamma', 0, 20),
        'missing': -999,
        'tree_method': 'gpu_hist',
        'objective': 'binary:logistic'
    }
    
    clf = xgb.XGBClassifier(**params)

    # fit for all folds and return composite utility score
    uts = []
    for i, (train_idx, valid_idx) in enumerate(cv.split(
        X,
        y,
        groups=groups)):
        
        train_data = X[train_idx, :], y.loc[:, 'action'].values[train_idx]
        valid_data = X[valid_idx, :], y.loc[:, 'action'].values[valid_idx]
        
        '''
        scaler.fit(X[train_idx, :])
        train_x_norm = scaler.transform(X[train_idx, :])

        pca = PCA()
        comp = pca.fit(train_x_norm)

        pca = PCA(n_components=70).fit(train_x_norm)
        train_x_transform = pca.transform(train_x_norm)

        valid_x_transform = pca.transform(scaler.transform(X[valid_idx, :]))
        '''
        
        _ = clf.fit(X[train_idx, :], y.loc[:, 'action'].values[train_idx])
        
        
        u = utility(clf, X[valid_idx, :], y.loc[:, 'date'].values[valid_idx], \
                    y.loc[:, 'weight'].values[valid_idx], y.loc[:, 'resp'].values[valid_idx])
        
        uts.append(u)
    
    print(f'Trial done: AUC values on folds: {uts}')
    return cv_fold_func(uts)

running cross validation

In [ ]:
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=30)
#You can increase n_trials parameter

In [ ]:
print('Best trial: score {}, params {}'.format(study.best_trial.value, study.best_trial.params))

In [ ]:
best_params = study.best_trial.params
best_params['tree_method'] = 'gpu_hist'      #gpu_hist is really fast
best_params['objective'] = 'binary:logistic'

In [ ]:
del dlabels, groups, X

In [ ]:
# Fit the XGBoost classifier with optimal hyperparameters
clf = xgb.XGBClassifier(**best_params)

In [ ]:
%time clf.fit(X_train, y.loc[:, 'action'])  #Used the whole training data

Submitting

In [ ]:
import janestreet
env = janestreet.make_env() # initialize the environment
iter_test = env.iter_test() # an iterator which loops over the test set

for (test_df, sample_prediction_df) in iter_test:
    X_test = test_df.loc[:, test_df.columns.str.contains('feature')]
    X_test.fillna(-999)
    y_preds = clf.predict(X_test)
    sample_prediction_df.action = y_preds
    env.predict(sample_prediction_df)